In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
%cd gdrive/MyDrive/TA\ Bareng\ Bareng/

/content/gdrive/.shortcut-targets-by-id/1mVs_asXEy-dFb5kdY3WcjA5GRJ2ZyRQ_/TA Bareng Bareng/Laporan Mingguan TA Harakan Julian Fausta/Minggu 15 (Final)/Ekstraksi


In [ ]:
!ls

 Benchmark			   'Hasil Anotasi'
 BRAINSTORMING.gdoc		   'Jurnal Fausta'
 Code				   'Laporan Mingguan TA Harakan Julian Fausta'
 Data				   'Laporan Skripsi'
'Diagram TA Bareng Bareng.drawio'  'Rekaman Bimbingan'


In [ ]:
kamus_list = ['18', '54', '68', '71', '89']

def get_index_kamus(input_path):
    for i in range(len(kamus_list)):
        if kamus_list[i] in input_path:
            return i
    return -1

In [ ]:
import pandas as pd

lookup_df = pd.read_csv('Laporan Mingguan TA Harakan Julian Fausta/Minggu 15 (Final)/Ekstraksi/12. Parallel Corpus - Spelling Checker/LookupIsFromIndonesia.csv')
print(lookup_df.shape)

(43, 2)


In [ ]:
def get_is_from_indonesia(input_path):
    for index, row in lookup_df.iterrows():
        if row['kamus'] in input_path:
            return True, row['is_from_indonesia']
    return False, -1

In [ ]:
def sentence_to_list(sentence):
    word_list = []
    word = ''
    for c in sentence.lower():
        if c.isalpha():
            word += c
        else:
            word_list.append(word)
            word = ''
    if word:
        word_list.append(word)
    return word_list

In [ ]:
import pandas as pd

def get_all_sentences(input_path):
    df = pd.read_csv(input_path)
    sentences = []
    success, is_from_indonesia = get_is_from_indonesia(input_path)

    if not success:
        return

    for index, row in df.iterrows():
        sentence = ''

        if is_from_indonesia:
            sentence = str(row['kalimat_asal']).lower()
        else:
            sentence = str(row['kalimat_tujuan']).lower()

        sentences.append(sentence)

    return sentences

In [ ]:
def get_is_from_indonesia_by_index(index_kamus):
    for index, row in lookup_df.iterrows():
        if kamus_list[index_kamus] in row['kamus']:
            return True, row['is_from_indonesia']
    return False, -1

In [ ]:
import pandas as pd

def get_gold_entries(input_path):
    df = pd.read_csv(input_path)
    gold_entries = []
    index_kamus = get_index_kamus(input_path)
    success, is_from_indonesia = get_is_from_indonesia_by_index(index_kamus)

    if not success:
        return []

    for index, row in df.iterrows():
        gold_entry = ''

        if is_from_indonesia:
            if not pd.isna(row['kalimat_asal']):
                gold_entry = str(row['kalimat_asal']).lower()
        else:
            if not pd.isna(row['kalimat_tujuan']):
                gold_entry = str(row['kalimat_tujuan']).lower()

        if len(gold_entry) > 1:
            gold_entries.append(gold_entry)

    return gold_entries

In [ ]:
import os

all_gold_entries = [[], [], [], [], []]
dir_path = 'Hasil Anotasi/'

for file in os.listdir(dir_path):
    input_path = dir_path + file
    print(input_path)
    index_kamus = get_index_kamus(input_path)
    all_gold_entries[index_kamus].extend(get_gold_entries(input_path))

for i in range(len(all_gold_entries)):
    print(len(all_gold_entries[i]))

Hasil Anotasi/GoldEntries_Tahap2_Niken - 18.csv
Hasil Anotasi/GoldEntries_Tahap2_Niken - 54.csv
Hasil Anotasi/GoldEntries_Tahap2_Niken - 68.csv
Hasil Anotasi/GoldEntries_Tahap2_Niken - 71.csv
Hasil Anotasi/GoldEntries_Tahap2_Niken - 89.csv
Hasil Anotasi/GoldEntries_Tahap2_Hafiz - 89.csv
Hasil Anotasi/GoldEntries_Tahap2_Hafiz - 71.csv
Hasil Anotasi/GoldEntries_Tahap2_Hafiz - 68.csv
Hasil Anotasi/GoldEntries_Tahap2_Hafiz - 54.csv
Hasil Anotasi/GoldEntries_Tahap2_Hafiz - 18.csv
Hasil Anotasi/GoldEntries_Tahap2_Qonita - 89.csv
Hasil Anotasi/GoldEntries_Tahap2_Qonita - 68.csv
0
98
99
95
96


In [ ]:
def is_same_sentence(gold_entry, sentence):
    gold_entry_list = sentence_to_list(gold_entry)
    sentence_list = sentence_to_list(sentence)

    if len(gold_entry_list) == len(sentence_list):
        for i in range(len(sentence_list)):
            if gold_entry_list[i] != sentence_list[i]:
                return False
        return True

    return False

In [ ]:
def evaluate(input_path):
    index_kamus = get_index_kamus(input_path)
    total_entries = len(all_gold_entries[index_kamus])

    if total_entries == 0:
        return []

    all_sentences = get_all_sentences(input_path)
    correct_sentences = 0
    correct_index = []

    for i in range(len(all_gold_entries[index_kamus])):
        gold_entry = all_gold_entries[index_kamus][i]
        found = False

        for sentence in all_sentences:
            if is_same_sentence(gold_entry, sentence):
                found = True
                break

        if found:
            correct_sentences += 1
            correct_index.append(i)

    print('correct sentences:', correct_sentences)
    print('total gold entries:', len(all_gold_entries[index_kamus]))
    print('accuracy:', correct_sentences * 100.0 / len(all_gold_entries[index_kamus]))
    return correct_index

In [ ]:
import os

parallel_corpus_path = 'Laporan Mingguan TA Harakan Julian Fausta/Minggu 15 (Final)/Ekstraksi/11. Parallel Corpus/'
correct_before = [[], [], [], [], []]

for file in os.listdir(parallel_corpus_path):
    if file[0].isnumeric() and file[-4:] == '.csv':
        index_kamus = get_index_kamus(file)
        if index_kamus != -1:
            print(file)
            input_path = parallel_corpus_path + file
            correct_before[index_kamus].extend(evaluate(input_path))

18_Parcor.csv
54_Parcor.csv
correct sentences: 16
total gold entries: 98
accuracy: 16.3265306122449
71_Parcor.csv
correct sentences: 81
total gold entries: 95
accuracy: 85.26315789473684
68_Parcor.csv
correct sentences: 58
total gold entries: 99
accuracy: 58.58585858585859
89_Parcor.csv
correct sentences: 86
total gold entries: 96
accuracy: 89.58333333333333


In [ ]:
import os

spelling_checker_path = 'Laporan Mingguan TA Harakan Julian Fausta/Minggu 15 (Final)/Ekstraksi/12. Parallel Corpus - Spelling Checker/'
correct_after = [[], [], [], [], []]

for file in os.listdir(spelling_checker_path):
    if file[0].isnumeric() and file[-4:] == '.csv':
        index_kamus = get_index_kamus(file)
        if index_kamus != -1:
            print(file)
            input_path = spelling_checker_path + file
            correct_after[index_kamus].extend(evaluate(input_path))

18_Parcor.csv
54_Parcor.csv
correct sentences: 18
total gold entries: 98
accuracy: 18.367346938775512
89_Parcor.csv
correct sentences: 86
total gold entries: 96
accuracy: 89.58333333333333
68_Parcor.csv
correct sentences: 58
total gold entries: 99
accuracy: 58.58585858585859
71_Parcor.csv
correct sentences: 81
total gold entries: 95
accuracy: 85.26315789473684


In [ ]:
for i in range(len(kamus_list)):
    total_entries = len(all_gold_entries[i])
    false_to_false = []
    false_to_true = []
    true_to_false = []
    true_to_true = []

    for j in range(total_entries):
        if j in correct_before[i]:
            if j in correct_after[i]:
                true_to_true.append(j)
            else:
                true_to_false.append(j)
        else:
            if j in correct_after[i]:
                false_to_true.append(j)
            else:
                false_to_false.append(j)

    print(kamus_list[i])
    print('false -> false:', len(false_to_false))
    print('false -> true:', len(false_to_true))
    print('true -> false:', len(true_to_false))
    print('true -> true:', len(true_to_true))

    for idx in false_to_true:
        print(all_gold_entries[i][idx])

    for idx in true_to_false:
        print(all_gold_entries[i][idx])

18
false -> false: 0
false -> true: 0
true -> false: 0
true -> true: 0
54
false -> false: 79
false -> true: 3
true -> false: 1
true -> true: 15
letakkan minuman itu di atas baki
kita tidak boleh memperlihatkan aurat kita
baca buku itu
mengasung itu sudah mati
68
false -> false: 41
false -> true: 0
true -> false: 0
true -> true: 58
71
false -> false: 14
false -> true: 0
true -> false: 0
true -> true: 81
89
false -> false: 10
false -> true: 0
true -> false: 0
true -> true: 86
